In [2]:
import pandas as pd
import numpy as np
import re
import json
from data_models import Movie,User,Rating
from connect_db import engine,base,Session
import requests

movies_df=pd.read_table('data/movies.dat','::',names=['title_year','genre'],index_col=0,engine='python')
ratings_df=pd.read_table('data/ratings.dat','::',names=['UserID','MovieID','rating','timestamp'],engine='python')
# users_df=pd.read_table('data/users.dat','::',names=['gender','age','occupation','zipcode'],index_col=0,engine='python')

In [3]:
movies_df

,title,genre
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy
...,...,...
3948,Meet the Parents (2000),Comedy
3949,Requiem for a Dream (2000),Drama
3950,Tigerland (2000),Drama
3951,Two Family House (2000),Drama


In [3]:
books_df=pd.read_csv('data/BX-Books.csv','\";\"',index_col=0,engine='python')

cols=books_df.columns.tolist()

cols[-1]='Image-URL-L'

books_df.columns=cols

books_df.index=[elem[1:] for elem in books_df.index]

books_df['Image-URL-L']=[elem[:-1] for elem in books_df['Image-URL-L']]

books_df.index.name='ISBN'

book_ratings_df=pd.read_csv('data/BX-Book-Ratings.csv','\";\"',index_col=0,engine='python')

book_ratings_df.columns=['ISBN','Book-Rating']

book_ratings_df.index=[int(elem[1:]) for elem in book_ratings_df.index]

book_ratings_df['Book-Rating']=[int(elem[:-1]) for elem in book_ratings_df['Book-Rating']]

book_ratings_df.index.name='User-ID'

book_users_df=pd.read_csv('data/BX-Users.csv','\";',index_col=0,engine='python')

book_users_df.index=[int(elem[1:]) for elem in book_users_df.index]

book_users_df.index.name='User-ID'

book_users_df.columns=['Location','Age']

book_users_df['Location']=[elem[1:] for elem in book_users_df['Location']]

book_users_df['Age'].iloc[0] is np.nan

book_users_df['Age']=[elem[1:-1] if elem is not np.nan else np.nan for elem in book_users_df['Age']]

In [137]:
movies_df.describe()

,title,genre
count,3883,3883
unique,3883,301
top,"Simple Twist of Fate, A (1994)",Drama
freq,1,843


In [146]:
book_ratings_df.describe()

,Book-Rating
count,1.149780e+06
mean,2.866950e+00
std,3.854184e+00
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,7.000000e+00
max,1.000000e+01


In [8]:
users_df.describe(include='all')

,gender,age,occupation,zip
count,6040,6040.000000,6040.000000,6040
unique,2,NaN,NaN,3439
top,M,NaN,NaN,48104
freq,4331,NaN,NaN,19
mean,NaN,30.639238,8.146854,NaN
std,NaN,12.895962,6.329511,NaN
min,NaN,1.000000,0.000000,NaN
25%,NaN,25.000000,3.000000,NaN
50%,NaN,25.000000,7.000000,NaN
75%,NaN,35.000000,14.000000,NaN


In [3]:
movies_df['year']=movies_df.title.str[-5:-1].astype(int)

In [34]:
movies_df.year

1       1995
2       1995
3       1995
4       1995
5       1995
        ... 
3948    2000
3949    2000
3950    2000
3951    2000
3952    2000
Name: year, Length: 3883, dtype: int64

In [4]:
movies_df['tt']=movies_df.title.str[:-7]

In [5]:
movies_df.drop('title',1)

,genre,year,tt
1,Animation|Children's|Comedy,1995,Toy Story
2,Adventure|Children's|Fantasy,1995,Jumanji
3,Comedy|Romance,1995,Grumpier Old Men
4,Comedy|Drama,1995,Waiting to Exhale
5,Comedy,1995,Father of the Bride Part II
...,...,...,...
3948,Comedy,2000,Meet the Parents
3949,Drama,2000,Requiem for a Dream
3950,Drama,2000,Tigerland
3951,Drama,2000,Two Family House


In [21]:
from data_models import Movie

In [14]:
import requests

In [15]:
omdb='http://www.omdbapi.com'

params={'i':'tt3896198','apikey':'f69c6afb','t':'Birthmarked','year':2018}

result=requests.get(omdb,params)

In [16]:
data=result.json()

In [17]:
data

{'Title': 'Birthmarked',
 'Year': '2018',
 'Rated': 'N/A',
 'Released': '30 Mar 2018',
 'Runtime': '90 min',
 'Genre': 'Comedy',
 'Director': 'Emanuel Hoss-Desmarais',
 'Writer': 'Marc Tulin, Marc Tulin (story), Emanuel Hoss-Desmarais (story)',
 'Actors': 'Matthew Goode, Toni Collette, Andreas Apergis, Jordan Poole',
 'Plot': 'Two scientists raise 3 children contrarily to their genetic tendencies to prove the ultimate power of nurture over nature.',
 'Language': 'English',
 'Country': 'Canada',
 'Awards': 'N/A',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTY3NzE5NzU2Nl5BMl5BanBnXkFtZTgwMTIzMzc4NDM@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '5.7/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '12%'},
  {'Source': 'Metacritic', 'Value': '44/100'}],
 'Metascore': '44',
 'imdbRating': '5.7',
 'imdbVotes': '2,273',
 'imdbID': 'tt4702826',
 'Type': 'movie',
 'DVD': 'N/A',
 'BoxOffice': 'N/A',
 'Production': 'Vertical Entertainment',
 'Website': 'N/A'

In [28]:
result.status_code

200

In [30]:
movies_df

,title,genre,year,tt
1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story
2,Jumanji (1995),Adventure|Children's|Fantasy,1995,Jumanji
3,Grumpier Old Men (1995),Comedy|Romance,1995,Grumpier Old Men
4,Waiting to Exhale (1995),Comedy|Drama,1995,Waiting to Exhale
5,Father of the Bride Part II (1995),Comedy,1995,Father of the Bride Part II
...,...,...,...,...
3948,Meet the Parents (2000),Comedy,2000,Meet the Parents
3949,Requiem for a Dream (2000),Drama,2000,Requiem for a Dream
3950,Tigerland (2000),Drama,2000,Tigerland
3951,Two Family House (2000),Drama,2000,Two Family House


In [3]:
from connect_db import engine

In [10]:
users_df.to_sql('user',engine,if_exists='append',index_label='id',method='multi')

In [4]:
conn=engine.connect()

In [12]:
users_df.shape

(6040, 4)

In [18]:
data[0]

KeyError: 0

In [20]:
data.keys()

dict_keys(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster', 'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production', 'Website', 'Response'])

In [23]:
data['Actors'].split()[0]

'Matthew'

In [29]:
movies_df_c=movies_df[900:1800].copy()

In [28]:
movies_df_c

,genre,year,title,title_len,genre_len
912,Drama|Romance|War,1942,Casablanca,10,17
913,Film-Noir|Mystery,1941,"Maltese Falcon, The",19,17
914,Musical|Romance,1964,My Fair Lady,12,15
915,Comedy|Romance,1954,Sabrina,7,14
916,Comedy|Romance,1953,Roman Holiday,13,14
...,...,...,...,...,...
1864,Comedy,1998,Sour Grapes,11,6
1865,Documentary,1998,Wild Man Blues,14,11
1866,Action|Comedy,1998,"Big Hit, The",12,13
1867,Action|Adventure,1998,Tarzan and the Lost City,24,16


In [7]:
a=[9]*10

In [8]:
movies_df_c['aaaa']=a

In [9]:
movies_df_c

,title_year,genre,aaaa
1,Toy Story (1995),Animation|Children's|Comedy,9
2,Jumanji (1995),Adventure|Children's|Fantasy,9
3,Grumpier Old Men (1995),Comedy|Romance,9
4,Waiting to Exhale (1995),Comedy|Drama,9
5,Father of the Bride Part II (1995),Comedy,9
6,Heat (1995),Action|Crime|Thriller,9
7,Sabrina (1995),Comedy|Romance,9
8,Tom and Huck (1995),Adventure|Children's,9
9,Sudden Death (1995),Action,9
10,GoldenEye (1995),Action|Adventure|Thriller,9


In [10]:
movies_df.shape

(3883, 2)

In [11]:
movies_df['year']=movies_df.title_year.str[-5:-1].astype(int)
movies_df['title']=movies_df.title_year.str[:-7]
movies_df.drop('title_year',1,inplace=True)

In [17]:
movies_df['title_len']=[len(title) for title in movies_df['title']]

In [18]:
movies_df.describe()

,year,title_len
count,3883.000000,3883.000000
mean,1986.066959,17.202678
std,16.895690,9.951110
min,1919.000000,1.000000
25%,1982.000000,10.000000
50%,1994.000000,15.000000
75%,1997.000000,21.000000
max,2000.000000,75.000000


In [21]:
for row in movies_df_c.iterrows():
    print(row[0])

11
12
13
14
15
16
17
18
19
20


In [22]:
movies_df_c

,genre,year,title,title_len
11,Comedy|Drama|Romance,1995,"American President, The",23
12,Comedy|Horror,1995,Dracula: Dead and Loving It,27
13,Animation|Children's,1995,Balto,5
14,Drama,1995,Nixon,5
15,Action|Adventure|Romance,1995,Cutthroat Island,16
16,Drama|Thriller,1995,Casino,6
17,Drama|Romance,1995,Sense and Sensibility,21
18,Thriller,1995,Four Rooms,10
19,Comedy,1995,Ace Ventura: When Nature Calls,30
20,Action,1995,Money Train,11


In [23]:
movies_df['genre_len']=[len(genre) for genre in movies_df['genre']]

In [24]:
movies_df.describe()

,year,title_len,genre_len
count,3883.000000,3883.000000,3883.000000
mean,1986.066959,17.202678,11.196498
std,16.895690,9.951110,6.843339
min,1919.000000,1.000000,3.000000
25%,1982.000000,10.000000,6.000000
50%,1994.000000,15.000000,9.000000
75%,1997.000000,21.000000,14.000000
max,2000.000000,75.000000,47.000000


In [2]:
session=Session()

In [3]:
q=session.query(Movie.id)

In [4]:
q

In [8]:
allids=[elem[0] for elem in q.all()]

In [32]:
type(allids)

list

In [33]:
ne_ratinngs_df=ratings_df[ratings_df.MovieID.isin(allids)]

In [35]:
ne_ratinngs_df.shape

(799164, 4)

In [36]:
ratings_df.shape

(1000209, 4)

In [37]:
allusers=[elem[0] for elem in session.query(User.id).all()]

In [38]:
nene_ratings_dr=ne_ratinngs_df[ne_ratinngs_df.UserID.isin(allusers)]

In [39]:
nene_ratings_dr.shape

(799164, 4)

In [4]:
book_users_df

,Location,Age
User-ID,,
1,"nyc, new york, usa",NaN
2,"stockton, california, usa",18
3,"moscow, yukon territory, russia",NaN
4,"porto, v.n.gaia, portugal",17
5,"farnborough, hants, united kingdom",NaN
...,...,...
278854,"portland, oregon, usa",NaN
278855,"tacoma, washington, united kingdom",50
278856,"brampton, ontario, canada",NaN


In [13]:
authors=list(set(books_df['Book-Author'].tolist()))

In [12]:
books_df.columns

Index(['Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [30]:
publishers=list(set(books_df.Publisher))

In [32]:
author_set=set(books_df['Book-Author'])

In [28]:
author_set.remove(np.nan)

In [ ]:
movies_df.Di